In [1]:
from objects import *
import networkx as nx
import pandas as pd

In [21]:
# Sets
nodes = [1, 2, 3]
arcs = [(1, 2), (2, 3), (3, 1), (2, 1), (3, 2), (1, 3)]
digraph = nx.DiGraph()
digraph.add_nodes_from(nodes)
digraph.add_edges_from(arcs)

traders = [1, 2]

# Parameters
loss_rate = 0.2
allowed_percentage = {node: 1 for node in nodes}
probability1 = 1
probability2 = 0.5
probability3 = 0.5
probability4 = 0.25
probability5 = 0.25
probability6 = 0.25
probability7 = 0.25

# Arc costs and capacities
arc_costs = {arc: 1 for arc in arcs}
arc_capacities = {arc: 25 for arc in arcs}

# Entry and exit costs
entry_costs1 = {node: 1 for node in nodes}
entry_costs2 = {node: 2 for node in nodes}
entry_costs3 = {node: 2 for node in nodes}
exit_costs1 = {node: 1 for node in nodes}
exit_costs2 = {node: 2 for node in nodes}
exit_costs3 = {node: 2 for node in nodes}

# Node capacities and demands
node_capacities = {node: 25 for node in nodes}
node_demands1 = None
node_demands2 = {1: 20, 2: 0, 3: 0}
node_demands3 = {1: 0, 2: 20, 3: 0}

# Production costs and capacities
production_costs1 = {1: 1, 2: 5, 3: 1}
production_costs2 = {1: 10, 2: 50, 3: 10}
production_costs3 = {1: 10, 2: 50, 3: 10}
production_capacities1 = {(1, 1): 0, (1, 2): 30, (1, 3): 30,
                          (2, 1): 0, (2, 2): 30, (2, 3): 30}
production_capacities2 = {(1, 1): 0, (1, 2): 30, (1, 3): 30,
                          (2, 1): 0, (2, 2): 30, (2, 3): 30} # t, n
production_capacities3 = {(1, 1): 30, (1, 2): 0, (1, 3): 30,
                          (2, 1): 30, (2, 2): 0, (2, 3): 30}

# Storage costs and capacities
storage_costs1 = {(1, 1): 1, (1, 2): 1, (1, 3): 1,
                  (2, 1): 1, (2, 2): 1, (2, 3): 1} # {(t, n): costs}
storage_costs2 = {(1, 1): 2, (1, 2): 2, (1, 3): 2,
                  (2, 1): 2, (2, 2): 2, (2, 3): 2}
storage_costs3 = {(1, 1): 2, (1, 2): 2, (1, 3): 2,
                  (2, 1): 2, (2, 2): 2, (2, 3): 2}
storage_capacities = {(1, 1): 100, (1, 2): 100, (1, 3): 100,
                      (2, 1): 100, (2, 2): 100, (2, 3): 100} # {(t, n): costs}

# Stages objects
# First stage
stage1 = Stage(1, arc_costs, entry_costs1, exit_costs1, arc_capacities, node_capacities, probability1, node_demands1, production_costs1, production_capacities1, storage_costs1, storage_capacities, None)

# Second stage
stage2 = Stage(2, arc_costs, entry_costs2, exit_costs2, arc_capacities, node_capacities, probability2, node_demands2, production_costs2, production_capacities1, storage_costs2, storage_capacities, stage1)
stage3 = Stage(3, arc_costs, entry_costs3, exit_costs3, arc_capacities, node_capacities, probability3, node_demands3, production_costs3, production_capacities2, storage_costs3, storage_capacities, stage1)

# Third stage: dummy for now
stage4 = Stage(4, arc_costs, entry_costs2, exit_costs2, arc_capacities, node_capacities, probability4, node_demands2, production_costs2, production_capacities1, storage_costs2, storage_capacities, stage2)
stage5 = Stage(5, arc_costs, entry_costs3, exit_costs3, arc_capacities, node_capacities, probability5, node_demands3, production_costs3, production_capacities2, storage_costs3, storage_capacities, stage2)
stage6 = Stage(6, arc_costs, entry_costs2, exit_costs2, arc_capacities, node_capacities, probability6, node_demands2, production_costs2, production_capacities1, storage_costs2, storage_capacities, stage3)
stage7 = Stage(7, arc_costs, entry_costs3, exit_costs3, arc_capacities, node_capacities, probability7, node_demands3, production_costs3, production_capacities2, storage_costs3, storage_capacities, stage3)


stages = [stage1, stage2, stage3, stage4, stage5, stage6, stage7]

# Problem object
problem = Problem(digraph, stages, traders, loss_rate, allowed_percentage)

# Production costs

In [22]:
production_df = pd.DataFrame([production_costs1, production_costs2, production_costs3]) #, columns = ["Scenario node 1", "Scenario node 2", "Scenario node 3"], index = ["Scenario node 1", "Scenario node 2", "Scenario node 3"])
production_df.columns = ["Scenario node 1", "Scenario node 2", "Scenario node 3"]
production_df.index = ["Node 1", "Node 2", "Node 3"]
production_df

,Scenario node 1,Scenario node 2,Scenario node 3
Node 1,1,5,1
Node 2,10,50,10
Node 3,10,50,10


# Production capacities

In [23]:
rows = []
for i, temp in enumerate([production_capacities1, production_capacities2, production_capacities3]):
    for k, v in temp.items():
        rows.append({"Scenario node": i+1, "Trader": k[0], "Node": k[1], "Capacity": v})
prod_cap_df = pd.DataFrame(rows)
prod_cap_df

,Scenario node,Trader,Node,Capacity
0,1,1,1,0
1,1,1,2,30
2,1,1,3,30
3,1,2,1,0
4,1,2,2,30
5,1,2,3,30
6,2,1,1,0
7,2,1,2,30
8,2,1,3,30
9,2,2,1,0


# Storage capacities

In [24]:
rows = []
for i, temp in enumerate([storage_capacities, storage_capacities, storage_capacities]):
    for k, v in temp.items():
        rows.append({"Scenario node": i+1, "Trader": k[0], "Node": k[1], "Capacity": v})
storage_cap_df = pd.DataFrame(rows)
storage_cap_df

,Scenario node,Trader,Node,Capacity
0,1,1,1,100
1,1,1,2,100
2,1,1,3,100
3,1,2,1,100
4,1,2,2,100
5,1,2,3,100
6,2,1,1,100
7,2,1,2,100
8,2,1,3,100
9,2,2,1,100


# Production costs

In [25]:
rows = []
for i, temp in enumerate([production_costs1, production_costs2, production_costs3]):
    for k, v in temp.items():
        rows.append({"Scenario node": i+1, "Node": k, "Costs": v})
prod_costs_df = pd.DataFrame(rows)
prod_costs_df

,Scenario node,Node,Costs
0,1,1,1
1,1,2,5
2,1,3,1
3,2,1,10
4,2,2,50
5,2,3,10
6,3,1,10
7,3,2,50
8,3,3,10


# Entry and exit costs

In [26]:
rows = []
for i, temp in enumerate([entry_costs1, entry_costs2, entry_costs3]):
    for k, v in temp.items():
        rows.append({"Scenario node": i+1, "Node": k, "Costs": v})
entry_costs_df = pd.DataFrame(rows)
entry_costs_df

,Scenario node,Node,Costs
0,1,1,1
1,1,2,1
2,1,3,1
3,2,1,2
4,2,2,2
5,2,3,2
6,3,1,2
7,3,2,2
8,3,3,2


In [27]:
rows = []
for i, temp in enumerate([exit_costs1, exit_costs2, exit_costs3]):
    for k, v in temp.items():
        rows.append({"Scenario node": i+1, "Node": k, "Costs": v})
exit_costs_df = pd.DataFrame(rows)
exit_costs_df

,Scenario node,Node,Costs
0,1,1,1
1,1,2,1
2,1,3,1
3,2,1,2
4,2,2,2
5,2,3,2
6,3,1,2
7,3,2,2
8,3,3,2


# Model running

In [28]:
model = problem.build_model()

In [29]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 23.4.0 23E214)

CPU model: Intel(R) Core(TM) i7-1060NG7 CPU @ 1.20GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 426 rows, 504 columns and 2442 nonzeros
Model fingerprint: 0x41f829ac
Coefficient statistics:
  Matrix range     [8e-01, 2e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+02]
Presolve removed 108 rows and 62 columns
Presolve time: 0.01s
Presolved: 318 rows, 442 columns, 2138 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.000000e+01   0.000000e+00      0s
     161    3.4375000e+02   0.000000e+00   0.000000e+00      0s

Solved in 161 iterations and 0.03 seconds (0.00 work units)
Optimal objective  3.437500000e+02


In [30]:
22.22222 * 3 + 20 # sales

86.66666000000001

# Flow variables

In [31]:
for t in problem.traders:
    for a in problem.arcs:
        for m in problem.stages:
            value = model.getVarByName(f'f[{t},{a[0]},{a[1]},{m.stage_id}]').X
            if value > 0:
                print(f"Trader {t}, Arc {a}, Stage {m.stage_id}: {value}")

Trader 2, Arc (2, 1), Stage 4: 20.0
Trader 2, Arc (3, 1), Stage 2: 25.0
Trader 2, Arc (3, 1), Stage 4: 5.0
Trader 2, Arc (3, 1), Stage 6: 25.0
Trader 2, Arc (3, 2), Stage 7: 25.0


# Storage variables

In [32]:
for t in problem.traders:
    for n in problem.nodes:
        for m in problem.stages:
            value = model.getVarByName(f'v[{t},{n},{m.stage_id}]').X
            if value > 0:
                print(f"Trader {t}, Node {n}, Stage {m.stage_id}: {value}")

Trader 2, Node 2, Stage 1: 20.0
Trader 2, Node 2, Stage 2: 20.0
Trader 2, Node 3, Stage 1: 25.0
Trader 2, Node 3, Stage 3: 25.0


# Production variables

In [33]:
for t in problem.traders:
    for n in problem.nodes:
        for m in problem.stages:
            value = model.getVarByName(f'q_production[{t},{n},{m.stage_id}]').X
            if value > 0:
                print(f"Trader {t}, Node {n}, Stage {m.stage_id}: {value}")

Trader 2, Node 2, Stage 1: 20.0
Trader 2, Node 3, Stage 1: 25.0
Trader 2, Node 3, Stage 4: 5.0


# Sales variables

In [34]:
for t in problem.traders:
    for n in problem.nodes:
        for m in problem.stages:
            value = model.getVarByName(f'q_sales[{t},{n},{m.stage_id}]').X
            if value > 0:
                print(f"Trader {t}, Node {n}, Stage {m.stage_id}: {value}")

Trader 2, Node 1, Stage 2: 20.0
Trader 2, Node 1, Stage 4: 20.0
Trader 2, Node 1, Stage 6: 20.0
Trader 2, Node 2, Stage 3: 20.0
Trader 2, Node 2, Stage 5: 20.0
Trader 2, Node 2, Stage 7: 20.0


# Entry variables

In [35]:
for t in problem.traders:
    for n in problem.nodes:
        for m in problem.stages:
            value = model.getVarByName(f'x_plus[{n},{m.stage_id},{t}]').X
            if value > 0:
                print(f"Trader {t}, Node {n}, Stage {m.stage_id}: {value}")

Trader 2, Node 2, Stage 1: 20.0
Trader 2, Node 3, Stage 1: 25.0


# Exit variables

In [36]:
for t in problem.traders:
    for n in problem.nodes:
        for m in problem.stages:
            value = model.getVarByName(f'x_minus[{n},{m.stage_id},{t}]').X
            if value > 0:
                print(f"Trader {t}, Node {n}, Stage {m.stage_id}: {value}")

Trader 2, Node 1, Stage 1: 20.0
Trader 2, Node 2, Stage 1: 20.0


# Sale of capacity variables

In [37]:
for t in problem.traders:
    for n in problem.nodes:
        for m in problem.stages:
            value = model.getVarByName(f'y_plus[{n},{m.stage_id},{t}]').X
            if value > 0:
                print(f"Trader {t}, Node {n}, Stage {m.stage_id}: {value}")

In [38]:
for t in problem.traders:
    for n in problem.nodes:
        for m in problem.stages:
            value = model.getVarByName(f'y_minus[{n},{m.stage_id},{t}]').X
            if value > 0:
                print(f"Trader {t}, Node {n}, Stage {m.stage_id}: {value}")

# Sanity check

In [39]:
for n in problem.nodes:
    for m in problem.stage_ids_star:
        lhs = 0
        rhs = 0
        for t in problem.traders:
            for a in problem.incoming_arcs[n]:
                lhs += (1 - problem.loss_rate) * model.getVarByName(f'f[{t},{a[0]},{a[1]},{m}]').X
            for a in problem.outgoing_arcs[n]:
                lhs -= model.getVarByName(f'f[{t},{n},{a[1]},{m}]').X
            lhs += model.getVarByName(f'q_production[{t},{n},{m}]').X
            rhs += model.getVarByName(f'x_minus[{n},{1},{t}]').X + model.getVarByName(f'x_minus[{n},{m},{t}]').X - model.getVarByName(f'y_minus[{n},{m},{t}]').X
        print(n, m, lhs, rhs)

1 2 20.0 20.0
1 3 0.0 20.0
1 4 20.0 20.0
1 5 0.0 20.0
1 6 20.0 20.0
1 7 0.0 20.0
2 2 0.0 20.0
2 3 0.0 20.0
2 4 -20.0 20.0
2 5 0.0 20.0
2 6 0.0 20.0
2 7 20.0 20.0
3 2 -25.0 0.0
3 3 0.0 0.0
3 4 0.0 0.0
3 5 0.0 0.0
3 6 -25.0 0.0
3 7 -25.0 0.0
